In [1]:
# importing dependencies 

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import json

In [42]:
#load in csv 
df = pd.read_excel('combined_fuction_inputs.xlsx')

In [24]:
df.head()

,company,start_URL,cookie,get_urls,get_title,get_body,get_meta,get_date
0,ABB Inc.,https://global.abb/group/en/media,"//a[@id=""btnApplyCheckboxSelection""]","//a[@class=""cmp-news-list__item__download""]",//article/header/h1,//div[@property='articleBody'],NaN,//aside[@class='oneabb-newsbank-news-Header-su...
1,AECOM Technology Corporation,https://aecom.com/press-releases/,"//a[@class=""aecom-global-cta aecom-global-cta-...",//h3[@class='pr-title']/a,//h2[@class='pr-single-title'],"//div[@class=""press-release-single-body-content""]",NaN,//p[@class='pr-single-date']
2,Argonne National Lab,https://www.anl.gov/news,NaN,//h3[@class='teaser__title']/a,//h1[@class='basic-header__title'],//div[@class='article-page__content'],/html/head/meta,//div[@class='article__byline']/span/div/time
3,"Cummins, Inc.",https://www.cummins.com/news,"//div[@id=""onetrust-accept-btn-container""]","//h3[@class=""field-content""]/a","//h1[@class=""node__title""]",//div[@class='node__content clearfix'],NaN,//div[@class='newsroom-date-author-wrapper']/s...
4,Central Florida Expressway Authority,https://www.cfxway.com/agency-information/news...,NaN,//h4[@class='cs-blog-title-inner']/a,//div[@class='cs-blog-title']/h2,//div[@class='cs-blog-content'],/html/head/meta,NaN


In [29]:
# # need to figure this out later. 

# import selenium
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')
# driver = webdriver.Chrome(options=options)
# driver.implicitly_wait(10)

In [36]:
# prep stuff

import selenium
from selenium import webdriver
driver = webdriver.Chrome()
driver.implicitly_wait(10)

In [7]:
import os
os.environ.get('Path')

'C:\\Users\\cobus\\anaconda3;C:\\Users\\cobus\\anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\cobus\\anaconda3\\Library\\usr\\bin;C:\\Users\\cobus\\anaconda3\\Library\\bin;C:\\Users\\cobus\\anaconda3\\Scripts;C:\\Users\\cobus\\anaconda3\\bin;C:\\Users\\cobus\\anaconda3\\condabin;C:\\Users\\cobus\\anaconda3;C:\\Users\\cobus\\anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\cobus\\anaconda3\\Library\\usr\\bin;C:\\Users\\cobus\\anaconda3\\Library\\bin;C:\\Users\\cobus\\anaconda3\\Scripts;C:\\windows\\system32;C:\\windows;C:\\windows\\System32\\Wbem;C:\\windows\\System32\\WindowsPowerShell\\v1.0;C:\\windows\\System32\\OpenSSH;C:\\Users\\cobus\\anaconda3\\Scripts;C:\\Users\\cobus\\AppData\\Local\\Microsoft\\WindowsApps;C:\\Users\\cobus\\AppData\\Local\\GitHubDesktop\\bin;C:\\Users\\cobus\\AppData\\Local\\Programs\\Microsoft VS Code\\bin;C:\\Users\\cobus\\Downloads\\chromedriver_win32;.'

In [37]:
def scrape(
    start_url, cookie, get_urls, get_title, get_body, get_meta, get_date
):
    
    records = []
    # Navigate to webpage, and click yes for cookies
    driver.get(start_url)
    # cookies
    if isinstance(cookie, str):
        try:
            driver.find_element('xpath', cookie).click()
        except Exception as e:
            print(f"CookieError: {e}, Website: {start_url}")


    # Find all of the anchor tags on the webpage
    links = driver.find_elements('xpath', get_urls)

    # Extract the URLs from the anchor tags
    urls = [link.get_attribute("href") for link in links]

    # Print the URLs if you'd like
    print(urls)

    # scraping body text and title for each article
    for url in urls:
        # need to do this. (check url, if there, ignore it)
        if url is None:
            continue
        try:
            driver.get(url)
            record = {'url': url}
        except selenium.common.exceptions.TimeoutException as e:
            print(e)
            continue
        
        try:
            title = driver.find_element('xpath',get_title).get_attribute('innerText')
            record['title'] = title
        except selenium.common.exceptions.NoSuchElementException as e:
            print(e)
            continue
        
        try:
            body = driver.find_element('xpath', get_body).get_attribute('innerText')
            record['body'] = body
        except Exception as e:
            print(f"GetBodyError: {e}, URL: {url}")
            
        if isinstance(get_meta, str):
            try:
                description = driver.find_element('xpath', get_meta).get_attribute('innerText')
                record['description'] = description
            except Exception as e:
                print(f"GetDescriptionError: {e}, URL: {url}")
        else:
            description = None
            
        if isinstance(get_date, str):
            try:
                date = driver.find_element('xpath', get_date).get_attribute('innerText')
                record['date'] = date
            except Exception as e:
                print(f"GetDateError: {e}, URL: {url}")
        else:
            date = None 
            
        records.append(record)
#         print(title)
    return records



In [43]:
rows = df.iloc[39:,:]

for index, row in rows.iterrows():
    # right here
    print(row['company'])
    scraped_articles = scrape(row['start_URL'], row['cookie'], row['get_urls'], row['get_title'], row['get_body'], row['get_meta'], row['get_date'])
    
    print(json.dumps(scraped_articles))

hypercraft
['https://hypercraftusa.com/funding-the-future-hypercraft-raises-6-5-million/', 'https://hypercraftusa.com/join-us-at-pri-2022/', 'https://hypercraftusa.com/sema-2022/', 'https://hypercraftusa.com/sema-launchpad-hypercraft-lifts-off-as-top-15-semi-finalist/', 'https://hypercraftusa.com/hypercraft-further-perfects-complete-ev-package-offerings-by-joining-forces-with-stealth-ev/', 'https://hypercraftusa.com/celebrating-the-community-performance-of-electric-vehicles/', 'https://hypercraftusa.com/ev-101-seminar-class-is-in-session/', 'https://hypercraftusa.com/on-the-two-guys-garage-podcast-hypercraft-talks-ev/', 'https://hypercraftusa.com/electrifying-the-marine-industry/', 'https://hypercraftusa.com/what-is-ehp-and-why-does-it-matter/']
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//h1[@class='entry-title']"}
  (Session info: chrome=111.0.5563.111)

[{"url": "https://hypercraftusa.com/funding-the-future-hypercraft-raises-6-5-million/", "tit

In [35]:
# Close the browser
driver.quit()

In [ ]:
    # make holding state
# def scrape
    # first line is `records = []`
    # on home page, click cookie and get urls
        # inside loop
        # `record = {}`
        # try visiting the page
        # if you succeed, add url to `record` (`record['url'] = url`)
        # try to get title and add it to `record`
        # try to get body and add it to `record`
        # try to get meta and add it to `record
        # finally, append `record` to `records`
    # return `records`
    

    
    

# If cross-refrencing in function

## option 1 
extra argument for new idea
connect to database
use keys to collect existing urls associated with comapanies already in database
turn that into a list
next step: 
pass that list as another argument to the big fuction 
create some sort of if statement in the big fuction that cross refrences urls and 

## option two 

connect to database in function and use get_item 




In [2]:
#load in csv 
df2 = pd.read_excel('lxml_inputs.xlsx')

In [3]:
df2.head()

,company,start_url,get_cookie,get_entry,get_title,get_url,get_date,get_summary
0,"Integrated Roadways, LLC",https://integratedroadways.com/news,NaN,//div[@class='np-article'],//h4,//a,//p[@class='np-credentials'],//p/span
1,ITSA (Intelligent Transportation Society of Am...,https://itsa.org/newsroom/,NaN,//div[@class='entry-content'],//h1,//a,//time,//p
2,Utah Inland Port Authority,https://inlandportauthority.utah.gov/category/...,NaN,//div[@class='entry-wrap'],//h2,//a,//time,//div[@class='entry-content excerpt']
3,Allison Transmission,https://ir.allisontransmission.com/news-and-ev...,NaN,//*[@class='1clearfix node node--nir-news--nir...,//div[@class='title-wrapper'],//a,//div[@class='nir-widget--field nir-widget--ne...,//div[@class='nir-widget--field nir-widget--ne...
4,OED,https://energy.utah.gov/about/news/,NaN,//div[@class='entry-wrap'],//h2,//a,NaN,NaN


In [4]:
# prep stuff

import selenium
from selenium import webdriver
driver = webdriver.Chrome()
driver.implicitly_wait(10)

In [5]:
from lxml import html

def special_scrape(
    start_url, get_cookie, get_entry, get_title, get_date, get_url, get_summary
    
):
    
    records = []
    # Navigate to webpage, and click yes for cookies
    try:
        driver.get(start_url)
    except selenium.common.exceptions.TimeoutException as e:
        print(e)
        return records
    # cookies
    if isinstance(get_cookie, str):
        try:
            driver.find_element('xpath', get_cookie).click()
        except Exception as e:
            print(f"CookieError: {e}, Website: {start_url}")


    # Find all entries
    entries = driver.find_elements('xpath',get_entry)
    

    # scraping body text and title for each article
    for entry in entries:
        tree = html.fromstring(entry.get_attribute('innerHTML'))
        record = {}
        try:
            title = tree.xpath(get_title)[0].text
            record['title']= title
            url = tree.xpath(get_url)[0].attrib['href']
            record['url']= url
        except:
            print('error')
            continue
        try:
            date = tree.xpath(get_date)[0].text
            record['date']= date
        except:
            pass 
        try:
            summary = tree.xpath(get_summary)[0].text
            record['summary']= summary
        except:
            pass 
    
    
            
        records.append(record)
#         print(title)
    return records



In [6]:
# rows = df2.iloc[3:,:]
for index, row in df2.iterrows():
    # right here
    print(row['company'])
    scraped_articles = special_scrape(row['start_url'], row['get_cookie'], row['get_entry'], row['get_title'], row['get_date'],row['get_url'], row['get_summary'])
    
    print(json.dumps(scraped_articles))

Integrated Roadways, LLC
[{"title": "IWCE 2023: Telecommunications technology expo highlights city solutions like smart pavement", "url": "https://www.americancityandcounty.com/2023/03/28/iwce-2023-enhanced-bandwidth-and-cybersecurity-in-the-face-of-natural-disasters/", "date": "American City & County - 3.30.2023", "summary": "Time is marching on and so is telecommunications technology \u2014 these days, at a breakneck pace.\u00a0"}, {"title": "This startup designed roads that pay for themselves (and charge your electric vehicle while driving)", "url": "https://integratedroadways.com/", "date": "Startland News - 3.8.2023", "summary": "Longtime Kansas City startup\u00a0"}, {"title": "Inductive Charging in Pavement: Meeting Needs of EVs", "url": "https://www.acpa.org/magazine/innovations-inductive-charging-in-pavement/", "date": "Concrete Pavement Progress - 1.18.2023", "summary": "As more electric vehicles are used by the general public and truck-based transportation companies, conversa

[{"title": "\n    ", "url": "https://inlandportauthority.utah.gov/2022/12/12/2022-economic-roundtable/", "date": null, "summary": "\n\n\n  "}, {"title": "\n    ", "url": "https://inlandportauthority.utah.gov/2022/12/09/terry-marasco-inland-port-management-promises-better-planning/", "date": null, "summary": "\n\n\n  "}, {"title": "\n    ", "url": "https://inlandportauthority.utah.gov/2022/12/04/https-slenterprise-com-index-php-news-latest-news-6250-uipa-s-flurry-of-activity-has-it-going-in-the-right-direction/", "date": null, "summary": "\n\n\n  "}, {"title": "\n    ", "url": "https://inlandportauthority.utah.gov/2022/11/17/states-test-an-electrifying-idea-roads-that-can-recharge-your-ev/", "date": null, "summary": "\n\n\n  "}, {"title": "\n    ", "url": "https://inlandportauthority.utah.gov/2022/11/15/dura-line-expands-salt-lake-city-utah-manufacturing-facilities/", "date": null, "summary": "\n\n\n  "}, {"title": "\n    ", "url": "https://inlandportauthority.utah.gov/2022/10/03/five-t

In [7]:
import re

ORD_ENDINGS = "st|nd|rd|th"

MONTH = "January|February|March|April|May|June|July|August|September|October|November|December|JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER"
MONTH_ABBREV = "Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC|June|July|Sept|SEPT"

#                  |   day     |   month            |  | year |
DAY_MONTH_YEAR = r"\d{{1,2}}\s+({}|({})\.?|\d{{1,2}})\s+\d{{4}}".format(MONTH, MONTH_ABBREV)
#                  month                    day               year
MONTH_DAY_YEAR = r"({}|({})\.?|\d{{1,2}})\s+\d{{1,2}}({})?,\s+\d{{4}}".format(MONTH, MONTH_ABBREV, ORD_ENDINGS)
#
YEAR_MONTH_DAY = r"\d{{4}}[-]".format()
#
ISOFORMAT = r"(\d{4})-(\d{2})-(\d{2})T(\d{2}):(\d{2}):(\d{2}(?:\.\d*)?)((-(\d{2}):(\d{2})|Z)?)"
#
NUMBERS_ONLY = r"(\d{1,2}[/ \.-]){2}(\d{2}|\d{4})"

PATTERN = re.compile(r"({})|({})|({})|({})".format(DAY_MONTH_YEAR, MONTH_DAY_YEAR, ISOFORMAT, NUMBERS_ONLY))

test_cases = [
    "01/01/2022",
    "1 February 2020",
    "1-1-95",
    "Feb 22, 2023",
    "February 14, 2023",
    "Published April 15, 2022",
    " February 22, 2023",
    " April 18, 2022 in ",
    "Kansas City, Mo., February 24, 2023 – Evergy, Inc. (NASDAQ: EVRG) today announced full year 2022 GAAP earnings of $753 million, or $3.27 per share, compared to GAAP earnings of $880 million, or $3.83 per share, for the full year 2021.&nbsp; Fourth quarter 2022 GAAP earnings were $7.5 million, or $0.03 per share, compared to earnings of $53.4 million, or $0.23 per share, for the fourth quarter of 2021.&nbsp;",
    "1.18.2020",
    "01.18.2020",
]

print(PATTERN)
for case in test_cases:
    matched = PATTERN.search(case)
    print(case, pd.to_datetime(matched.group(0)))

re.compile('(\\d{1,2}\\s+(January|February|March|April|May|June|July|August|September|October|November|December|JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER|(Jan|Feb|Mar|)
01/01/2022 2020-01-01 00:00:00
1 February 2020 2020-02-01 00:00:00
1-1-95 1995-01-01 00:00:00
Feb 22, 2023 2023-02-22 00:00:00
February 14, 2023 2023-02-14 00:00:00
Published April 15, 2022 2022-04-15 00:00:00
 February 22, 2023 2023-02-22 00:00:00
 April 18, 2022 in  2022-04-18 00:00:00
Kansas City, Mo., February 24, 2023 – Evergy, Inc. (NASDAQ: EVRG) today announced full year 2022 GAAP earnings of $753 million, or $3.27 per share, compared to GAAP earnings of $880 million, or $3.83 per share, for the full year 2021.&nbsp; Fourth quarter 2022 GAAP earnings were $7.5 million, or $0.03 per share, compared to earnings of $53.4 million, or $0.23 per share, for the fourth quarter of 2021.&nbsp; 2023-02-24 00:00:00
1.18.2020 2020-01-18 00:00:00
01.18.2020 2020-01-18 00:00:00


Ohio Turnpike

In [15]:
import selenium
from selenium import webdriver
driver = webdriver.Chrome()
driver.implicitly_wait(10)

In [10]:
def Ohio_scrape(
    
    
):
    start_url = 'https://www.ohioturnpike.org/media/news-releases' 
    get_urls = "//h2//a"
    records = []
    
    try:
        driver.get(start_url)
    except selenium.common.exceptions.TimeoutException as e:
        print(e)
        
 # Find all of the anchor tags on the webpage
    links = driver.find_elements('xpath', get_urls)
   
    # Extract the URLs from the anchor tags
    urls = [link.get_attribute("href") for link in links]
    
    titles = [link.get_attribute("text") for link in links]
    for url, title in zip(urls, titles):
        records.append({'URL': url, 'Title': title})
    
            
    return records



In [11]:



scraped_titles = Ohio_scrape()
    
print(json.dumps(scraped_titles))

[{"URL": "https://www.ohioturnpike.org/docs/default-source/news-release-documents/news-releases-2023/ohio-turnpike-announces-detour-at-toll-plaza-118-(sandusky-norwalk)u-s-250-exit-for-pavement-resurfacing-of-ramps.pdf?sfvrsn=d6d2fdc4_2", "Title": "Ohio Turnpike Announces Resurfacing of Ramps at\u00a0Toll Plaza 118 (Sandusky-Norwalk/US 250)"}, {"URL": "https://www.ohioturnpike.org/docs/default-source/news-release-documents/news-releases-2023/ohio-turnpike-issues-detour-from-milepost-39-to-52-in-fulton-and-lucas-counties.pdf?sfvrsn=66a1fdc4_2", "Title": "Ohio Turnpike Issues Detour from Milepost 39 to 52 in Fulton and Lucas Counties"}, {"URL": "https://www.ohioturnpike.org/docs/default-source/news-release-documents/news-releases-2023/ohio-turnpike-issues-weather-event-travel-restriction-for-high-profile-vehicles-03-24-23.pdf?sfvrsn=6da5fdc4_2", "Title": "Ohio Turnpike Issues Weather Event Travel Restriction for High-Profile Vehicles\n\n"}, {"URL": "https://www.ohioturnpike.org/docs/defa

In [18]:
def Utah_clean_scrape(
    
    
):
    start_url = 'https://utahcleancities.org/press-releases/' 
    get_urls = "//h3/a"
    get_time = '//time'
    records = []
    
    try:
        driver.get(start_url)
    except selenium.common.exceptions.TimeoutException as e:
        print(e)
        
 # Find all of the anchor tags on the webpage
    links = driver.find_elements('xpath', get_urls)
   
    # Extract the URLs from the anchor tags
    urls = [link.get_attribute("href") for link in links]
            
    
    titles = [link.get_attribute("text") for link in links]
    
    dates = driver.find_elements('xpath', get_time)
    date = [d.get_attribute("datetime") for d in dates]
    for url, title, d in zip(urls, titles, date):
    
        records.append({'URL': url, 'Title': title, 'Date': d})
    
            
    return records



In [19]:
scraped_titles = Utah_clean_scrape()
    
print(json.dumps(scraped_titles))

[{"URL": "https://utahcleancities.org/doe-press-release-doe-launches-new-50-million-program-to-help-communities-meet-their-clean-energy-goals/", "Title": "DOE PRESS RELEASE: DOE Launches New $50 Million Program To Help Communities Meet Their Clean Energy Goals", "Date": "2023-01-19T10:13:57-06:00"}, {"URL": "https://utahcleancities.org/recently-released-a-joint-strategy-to-transform-transportation/", "Title": "Recently Released: A Joint Strategy to Transform Transportation", "Date": "2023-01-12T13:18:56-06:00"}, {"URL": "https://utahcleancities.org/doe-announces-upcoming-funding-est-february-2023/", "Title": "DOE Announces Upcoming Funding (est. February 2023)", "Date": "2023-01-04T17:15:36-06:00"}, {"URL": "https://utahcleancities.org/clean-cities-hall-of-fame-2022/", "Title": "U.S. Department of Energy Honors Utah Leader for Advancing Alternative Fuels and Transportation Technologies", "Date": "2022-12-04T18:27:35-06:00"}, {"URL": "https://utahcleancities.org/14th-annual-advanced-fue